In [23]:
import matplotlib.pyplot as plt

# FrameWork utilizado --> Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#
import tqdm

# Pegando o Dataset pelo Keras
from keras.datasets import mnist

In [24]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# DataLoader
O data Loader será utilizado para poder pegar os dados aleatoriamente e controlar o "Batch Size"

In [25]:
class MNIST(Dataset):
    
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self,idx):
        return self.X[idx],self.y[idx]
    

In [26]:
train_dl = DataLoader(MNIST(train_X, train_y), batch_size=256)
test_dl = DataLoader(MNIST(test_X, test_y), batch_size= 64)

# Criando o Modelo de MLP

In [27]:

class MLP_Model(nn.Module):
    
    def __init__(self):
        super(MLP_Model,self).__init__()
        
        # nn.Linear(a,b) --> a entrada, b mapeia ?
        # O primeiro neurônio tem 4 no parênteses pois o data set tem 4 caracteristicas no X,  
        
        self.L1 =  nn.Linear(4,32)
        self.L2 =  nn.Linear(32,16)
        self.L3 =  nn.Linear(16,8)
        self.L4 =  nn.Linear(8,3) 
        
        self.A1 =  nn.ReLU()
        
        # No segundo, receberá o tanto que saiu na layer passada, logo:
        # nn.Linear(c,d) --> c == a, d == quantidasde que sera y previsto, ex: probabilidade de ser cada uma das flores
        
        self.L2 =  nn.Linear(32,16)
        self.L3 =  nn.Linear(16,8)
        self.L4 =  nn.Linear(8,3) 
        
        # SoftMax é uma função para classificação

        self.A2 = nn.Softmax()
        
    def forward(self, x):
        
        
        
        x = self.L1(x)
        # Aqui estou transformando o vetor inicial de 4 em um vetor de 
        x = self.A1(x)
        # Escalando o vetor via função sigmoid, outra possivel é a função reLU --> ler mais sobre
        x = self.L2(x)
        x = self.A1(x)
        x = self.L3(x)
        x = self.A1(x)
        x = self.L4(x)
        # Aqui estou transformando o vetor modificado de tamanho 5 em um vetor de 3
        x = self.A2(x)
        
        
        
        return x       

# Treinando o modelo


In [28]:
def training( N_Epochs, model, loss_fn, opt  ):
    
    loss_list = []
    
    for epoch in tqdm(range(N_Epochs+1)):
        for xb, yb in train_dl:
            
            y_pred = model(xb.float())
            loss   = loss_fn(y_pred, yb.long())
            
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        
        #print(epoch, "loss=", loss.item())
        loss_list.append(loss.item())
            
    
    plt.figure(figsize = (14,6))
    plt.title("cost decay")
    plt.plot(loss_list)
    plt.xlabel("epoch")
    plt.ylabel("cost")
    
    
    
    